# Data 불러오기

In [1]:
import os

import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import datetime as dt
from sqlalchemy import create_engine, event
import functools


def disjunc(*conditions):
    return functools.reduce(np.logical_or, conditions)


pd.set_option("display.float_format", "{:.2f}".format)
pd.set_option("display.max_columns", None)

In [2]:

%store -r master
%store -r mas_fil
%store -r fin_mas
%store -r check_qty
%store -r id_monthly_sales
%store -r user_3month_list
%store -r size_chk
%store -r sales_size #2111_2402
# %store -r sales_size_screen

no stored variable or alias #2111_2402


In [3]:
sorted(sales_size.YYYYMM.unique())

['202111',
 '202112',
 '202201',
 '202202',
 '202203',
 '202204',
 '202205',
 '202206',
 '202207',
 '202208',
 '202209',
 '202210',
 '202211',
 '202212',
 '202301',
 '202302',
 '202303',
 '202304',
 '202305',
 '202306',
 '202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312',
 '202401',
 '202402']

# Base Dataframe 생성

### 1. Total Tobacco Purchaser

### Outlier 

In [4]:
check_outlier = sales_size[(sales_size["sku"] >= 11) | (sales_size.qty_sum >= 61.0)]
check_outlier

,id,YYYYMM,CIGATYPE,ProductFamilyCode,ProductSubFamilyCode,ENGNAME,qty,sku,qty_sum
59,00002353C21A9F6A835A2A3E690A58848E771D60E82C8A...,202208,CC,BOHEM,BOHEMSS,BOHEM CIGAR ICE FIT,2.00,12,15.00
60,00002353C21A9F6A835A2A3E690A58848E771D60E82C8A...,202208,CC,DUNHILL,DUNFC,DUNHILL FINECUT 0.1MG,1.00,12,15.00
61,00002353C21A9F6A835A2A3E690A58848E771D60E82C8A...,202208,CC,DUNHILL,DUNFC,DUNHILL FINECUT 1MG,1.00,12,15.00
62,00002353C21A9F6A835A2A3E690A58848E771D60E82C8A...,202208,CC,DUNHILL,DUNHILLKS,DUNHILL 6MG,1.00,12,15.00
63,00002353C21A9F6A835A2A3E690A58848E771D60E82C8A...,202208,CC,ESSE,ESSE,ESSE GOLDEN LEAF,2.00,12,15.00
...,...,...,...,...,...,...,...,...,...
10234098,FFF3B969EF54F9D4F5D480E6AFA992EA12673422A22FEE...,202303,CC,MTHALLA,MTHALLA,MT. HALLA DELUXE,5.00,11,23.00
10234099,FFF3B969EF54F9D4F5D480E6AFA992EA12673422A22FEE...,202303,CC,RAISON,RASIONFR,RAISON FRENCH BLACK,3.00,11,23.00
10234100,FFF3B969EF54F9D4F5D480E6AFA992EA12673422A22FEE...,202303,CC,THEONE,THEONE,THE ONE ORANGE,2.00,11,23.00
10234101,FFF3B969EF54F9D4F5D480E6AFA992EA12673422A22FEE...,202303,CC,THIS,THISPLUS,THIS PLUS,1.00,11,23.00


### Total Valid Tobacco Purchaser = 3 month user update

In [14]:
month = "202401"

In [6]:
datelist = sales_size_screen.set_index("YYYYMM")  # 인덱스를 YYYYMM 열로 설정

In [7]:
months = datelist.index.unique().sort_values()
months

Index(['202111', '202112', '202201', '202202', '202203', '202204', '202205',
       '202206', '202207', '202208', '202209', '202210', '202211', '202212',
       '202301', '202302', '202303', '202304', '202305', '202306', '202307',
       '202308', '202309', '202310', '202311', '202312', '202401'],
      dtype='object', name='YYYYMM')

In [8]:
monthlist = [[a, b, c] for a, b, c in zip(months[3:], months[:-3], months[2:-1])]
monthlist

[['202202', '202111', '202201'],
 ['202203', '202112', '202202'],
 ['202204', '202201', '202203'],
 ['202205', '202202', '202204'],
 ['202206', '202203', '202205'],
 ['202207', '202204', '202206'],
 ['202208', '202205', '202207'],
 ['202209', '202206', '202208'],
 ['202210', '202207', '202209'],
 ['202211', '202208', '202210'],
 ['202212', '202209', '202211'],
 ['202301', '202210', '202212'],
 ['202302', '202211', '202301'],
 ['202303', '202212', '202302'],
 ['202304', '202301', '202303'],
 ['202305', '202302', '202304'],
 ['202306', '202303', '202305'],
 ['202307', '202304', '202306'],
 ['202308', '202305', '202307'],
 ['202309', '202306', '202308'],
 ['202310', '202307', '202309'],
 ['202311', '202308', '202310'],
 ['202312', '202309', '202311'],
 ['202401', '202310', '202312']]

# Database Structure 

In [8]:
user_3month_list.groupby("YYYYMM").nunique()

,id
YYYYMM,
202110,143187
202111,147027
202112,145930
202201,140434
202202,132780
202203,143346
202204,151228
202205,162597
202206,167654


In [16]:
sales_size_screen = sales_size[(sales_size["sku"] < 11) & (sales_size.qty_sum < 61.0)]

In [19]:
months = ["202309", "202310", "202311", "202312", "202401", "202402"]
data = {"title": ["Month", "Total", "valid", "screening", "outliers"], "numbers": []}

for month in months:
    cur_only_screen = sales_size_screen.merge(
        user_3month_list[user_3month_list["YYYYMM"] == month],
        how="inner",
        on=["YYYYMM", "id"],
    )
    numbers = [
        month,
        len(id_monthly_sales[id_monthly_sales.YYYYMM == month].id.unique()),  # Total
        len(user_3month_list[user_3month_list.YYYYMM == month]),  # valid
        len(sales_size_screen[sales_size_screen.YYYYMM == month].id.unique())
        - len(cur_only_screen.id.unique()),  # screening
        len(sales_size[sales_size.YYYYMM == month].id.unique())
        - len(
            sales_size_screen[sales_size_screen.YYYYMM == month].id.unique()
        ),  # outliers
    ]
    data["numbers"].append(numbers)

data = pd.DataFrame(data["numbers"], columns=data["title"])
data

,Month,Total,valid,screening,outliers
0,202309,287975,209723,77093,1159
1,202310,289933,212087,76751,1095
2,202311,277086,205452,70558,1076
3,202312,276726,202772,72899,1055
4,202401,271554,197372,73178,1004
5,202402,267616,194830,71841,945


# Tobacco Pack Purchase Universe (Pack Volume Base)
## 2-1. Tobacco Type Share : Volume Base

In [20]:
month_id = user_3month_list[user_3month_list.YYYYMM == month]["id"]
month_history = sales_size[
    (sales_size.id.isin(month_id)) & (sales_size.YYYYMM == month)
]

month_history

,id,YYYYMM,CIGATYPE,ProductFamilyCode,ProductSubFamilyCode,ENGNAME,qty,sku,qty_sum
6,00000496C27842128286B531DE109451C6C2C786A075F1...,202402,CC,DUNHILL,DUNHILLKS,DUNHILL 1MG 14,16.10,1,16.10
100,000070AEA35BE51A78AC975475425801DBE6960F193C63...,202402,CC,DUNHILL,DUNHILLKS,DUNHILL 1MG,16.00,1,16.00
227,0000EA3428F8591B1477806D69837187D9B98F7998A236...,202402,CC,THIS,THISPLUS,THIS PLUS,1.00,1,1.00
270,000135D0670ECB38AF0A6DCFAF9F45B05569C8B04C2F08...,202402,CC,ESSE,ESSEC,ESSE CHANGE 1MG,2.00,1,2.00
283,0001506CB7DC72B8648B876A81CBC6762971AA9F6ABA5C...,202402,CC,MLB,MLBFRESH,MARLBORO WHITE FRESH,2.00,1,2.00
...,...,...,...,...,...,...,...,...,...
10235601,FFFDBB36F0379949C6CC04FF64D47287BA52E91B30BF5B...,202402,CC,ESSE,ESSEC,ESSE CHANGE 1MG,1.00,2,8.00
10235602,FFFDBB36F0379949C6CC04FF64D47287BA52E91B30BF5B...,202402,HnB,IQOS,TEREA,TEREA AMBER,7.00,2,8.00
10235611,FFFDCB52F414FF382D443CCAED4A9D1405B4C2091B8873...,202402,CC,PLT,PLTKS,PARLIAMENT HYBRID 5,6.00,1,6.00
10235726,FFFF9C2BB80D17CDFD68737D3C63A13B66F022ED9F8654...,202402,CC,RAISON,RAISON,RAISON BLUE,3.00,1,3.00


In [21]:
month_history.qty.sum()
# 23년 10월 총 구매 갑 수 - 102만6681갑
# 24년 1월 총 구매 갑 수 - 95만6069갑
# 24년 2월 총 구매 갑 수 - 90만5388갑

905388.6

In [22]:
month_list = sorted(sales_size.YYYYMM.unique())


month_list

['202111',
 '202112',
 '202201',
 '202202',
 '202203',
 '202204',
 '202205',
 '202206',
 '202207',
 '202208',
 '202209',
 '202210',
 '202211',
 '202212',
 '202301',
 '202302',
 '202303',
 '202304',
 '202305',
 '202306',
 '202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312',
 '202401',
 '202402']

In [24]:
month_list[15:]

['202209',
 '202210',
 '202211',
 '202212',
 '202301',
 '202302',
 '202303',
 '202304',
 '202305',
 '202306',
 '202307',
 '202308',
 '202309',
 '202310',
 '202311',
 '202312',
 '202401',
 '202402']

In [25]:
Tobbacco_pack_df = pd.DataFrame(columns=["Month", "CC_sum", "HnB_sum"])


for month in month_list:
    month_id = user_3month_list[user_3month_list.YYYYMM == month]["id"]
    month_history = check_qty[
        (check_qty.id.isin(month_id)) & (check_qty.YYYYMM == month)
    ]
    CC_qty = month_history[month_history["CIGATYPE"] == "CC"].qty.sum()
    HnB_qty = month_history[month_history["CIGATYPE"] == "HnB"].qty.sum()
    Tobbacco_pack_df = pd.concat(
        [
            Tobbacco_pack_df,
            pd.DataFrame({"Month": [month], "CC_sum": [CC_qty], "HnB_sum": [HnB_qty]}),
        ]
    )

Tobbacco_pack_df

C:\Users\gchoi4\AppData\Local\Temp\ipykernel_4120\897802752.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Type_share_volume_df = pd.concat(


,Month,CC_sum,HnB_sum
0,202111,551074.60,142414.00
0,202112,542910.80,147433.00
0,202201,507815.50,143256.00
0,202202,455928.70,131787.00
0,202203,545219.10,150838.00
0,202204,571334.10,154258.00
0,202205,624102.40,166437.00
0,202206,625262.30,168683.00
0,202207,656776.40,172168.00
0,202208,629947.20,164235.00


In [19]:
# Type_share_volume_df.to_excel("Tobacco Purchasers Universe_0319.xlsx")

# Tobacco Purchasers Universe
## 2-2. Tobacco Type Share : User Base

In [26]:
Type_share_user_dict = {}

for month in month_list:
    month_id = user_3month_list[user_3month_list.YYYYMM == month]["id"]
    month_history = check_qty[
        (check_qty.id.isin(month_id)) & (check_qty.YYYYMM == month)
    ]
    month_user_cigatype = month_history[["id", "CIGATYPE", "Company"]]
    # "Company" 컬럼의 공백 제거
    month_user_cigatype = month_user_cigatype.copy()
    month_user_cigatype["Company"] = month_user_cigatype["Company"].str.strip()

    # "Company" 컬럼이 "PMK"가 아닌 경우 "Comp"로 대체
    month_user_cigatype["Company"] = np.where(
        month_user_cigatype["Company"] != "PMK", "Comp", month_user_cigatype["Company"]
    )

    month_user_cigatype["retype"] = np.where(
        month_user_cigatype["CIGATYPE"] == "CC",
        "CC",
        month_user_cigatype["Company"] + " " + month_user_cigatype["CIGATYPE"],
    )

    # "id"를 기준으로 그룹화하여 "retype" 컬럼의 고유한 값들을 리스트로 저장
    month_user_cigatype = (
        month_user_cigatype[["id", "retype"]].groupby("id").agg(["unique"])
    )

    # 컬럼명을 "retype"으로 변경
    month_user_cigatype.columns = ["retype"]

    # "retype" 컬럼의 리스트 값을 정렬하고 문자열로 조합하여 쉼표로 구분되는 형식으로 변경
    month_user_cigatype["retype"] = (
        month_user_cigatype["retype"]
        .apply(lambda x: sorted(x, reverse=True))  # 내림차순으로 정렬
        .apply(lambda x: (",").join(x))
    )

    month_user_cigatype_n = month_user_cigatype.groupby("retype").size().reset_index()
    month_user_cigatype_n.columns = ["retype", "N"]
    month_user_cigatype_n["perc"] = (
        (month_user_cigatype_n["N"]) / (month_user_cigatype_n["N"].sum(axis=0)) * 100
    )

    # 결과를 데이터프레임에 추가
    Type_share_user_dict[month] = month_user_cigatype_n

Type_share_user_dict

{'202111':                 retype       N  perc
 0                   CC  113344 77.09
 1             Comp HnB   11450  7.79
 2          Comp HnB,CC    7500  5.10
 3              PMK HnB    9201  6.26
 4           PMK HnB,CC    4477  3.05
 5     PMK HnB,Comp HnB     594  0.40
 6  PMK HnB,Comp HnB,CC     461  0.31,
 '202112':                 retype       N  perc
 0                   CC  111486 76.40
 1             Comp HnB   11861  8.13
 2          Comp HnB,CC    7757  5.32
 3              PMK HnB    9391  6.44
 4           PMK HnB,CC    4304  2.95
 5     PMK HnB,Comp HnB     614  0.42
 6  PMK HnB,Comp HnB,CC     517  0.35,
 '202201':                 retype       N  perc
 0                   CC  106057 75.52
 1             Comp HnB   12447  8.86
 2          Comp HnB,CC    7700  5.48
 3              PMK HnB    8991  6.40
 4           PMK HnB,CC    4174  2.97
 5     PMK HnB,Comp HnB     569  0.41
 6  PMK HnB,Comp HnB,CC     496  0.35,
 '202202':                 retype      N  perc
 0      

In [35]:
# a = result_dict["202311"]
# b = result_dict["202312"]
c = Type_share_user_dict["202402"]

# d = pd.concat([a, b, c])
c.to_excel("Tobacco Type Share_User Base_202402.xlsx")
# d

# 3-1. Demographics % | Age 

### TOTAL

In [23]:
demo_history = month_history[["id", "gender", "age"]]
print(len(demo_history.id.unique()))
demo_history = demo_history.drop_duplicates()
print(len(demo_history.id.unique()))

197372
197372


In [24]:
age_demo = demo_history.age.value_counts().sort_index()
age_demo = pd.DataFrame(age_demo).reset_index()
age_demo["%"] = age_demo["count"] / age_demo["count"].sum() * 100
age_demo
# age["%"] = (age / age.sum()
# age.to_frame()

,age,count,%
0,20대,31584,16.00
1,30대,43357,21.97
2,40대,62715,31.78
3,50대,44095,22.34
4,60대,12676,6.42
5,70대,2013,1.02
6,미상,932,0.47


In [25]:
gender_demo = demo_history.gender.value_counts().sort_index()
gender_demo = pd.DataFrame(gender_demo).reset_index()
gender_demo["%"] = gender_demo["count"] / gender_demo["count"].sum() * 100
gender_demo["count"].sum()
gender_demo
# gender["%"] = (gender / gender.sum()
# gender.to_frame()

,gender,count,%
0,남,132467,67.12
1,미상,86,0.04
2,여,64819,32.84


### HNB

In [39]:
month_id = user_3month_list[user_3month_list.YYYYMM == month]["id"]
month_hnb_history = check_qty[
    (check_qty.id.isin(month_id))
    & (check_qty.YYYYMM == month)
    & (check_qty.CIGATYPE == "HnB")
]

demo_hnb_history = month_hnb_history[["id", "gender", "age"]]
print(len(demo_hnb_history.id.unique()))
demo_hnb_history = demo_hnb_history.drop_duplicates()
print(len(demo_hnb_history.id.unique()))

55422
55422


In [40]:
age_hnb_demo = demo_hnb_history.age.value_counts().sort_index()
age_hnb_demo = pd.DataFrame(age_hnb_demo).reset_index()
age_hnb_demo["%"] = age_hnb_demo["count"] / age_hnb_demo["count"].sum() * 100
age_hnb_demo
# age["%"] = (age / age.sum()
# age.to_frame()

,age,count,%
0,20대,9711,17.52
1,30대,14640,26.42
2,40대,19177,34.60
3,50대,9681,17.47
4,60대,1714,3.09
5,70대,306,0.55
6,미상,193,0.35


In [41]:
gender_hnb_demo = demo_hnb_history.gender.value_counts().sort_index()
gender_hnb_demo = pd.DataFrame(gender_hnb_demo).reset_index()
gender_hnb_demo["%"] = gender_hnb_demo["count"] / gender_hnb_demo["count"].sum() * 100
gender_hnb_demo["count"].sum()
gender_hnb_demo
# gender["%"] = (gender / gender.sum()
# gender.to_frame()

,gender,count,%
0,남,37443,67.56
1,미상,15,0.03
2,여,17964,32.41


### CC

In [42]:
month_id = user_3month_list[user_3month_list.YYYYMM == month]["id"]
month_CC_history = check_qty[
    (check_qty.id.isin(month_id))
    & (check_qty.YYYYMM == month)
    & (check_qty.CIGATYPE == "CC")
]

demo_CC_history = month_CC_history[["id", "gender", "age"]]
print(len(demo_CC_history.id.unique()))
demo_CC_history = demo_CC_history.drop_duplicates()
print(len(demo_CC_history.id.unique()))

159059
159059


In [43]:
age_CC_demo = demo_CC_history.age.value_counts().sort_index()
age_CC_demo = pd.DataFrame(age_CC_demo).reset_index()
age_CC_demo["%"] = age_CC_demo["count"] / age_CC_demo["count"].sum() * 100
age_CC_demo
# age["%"] = (age / age.sum()
# age.to_frame()

,age,count,%
0,20대,25921,16.30
1,30대,33886,21.30
2,40대,48800,30.68
3,50대,36567,22.99
4,60대,11191,7.04
5,70대,1823,1.15
6,미상,871,0.55


In [44]:
gender_CC_demo = demo_CC_history.gender.value_counts().sort_index()
gender_CC_demo = pd.DataFrame(gender_CC_demo).reset_index()
gender_CC_demo["%"] = gender_CC_demo["count"] / gender_CC_demo["count"].sum() * 100
gender_CC_demo["count"].sum()
gender_CC_demo
# gender["%"] = (gender / gender.sum()
# gender.to_frame()

,gender,count,%
0,남,107262,67.44
1,미상,65,0.04
2,여,51732,32.52


# 5-1. HNB Category | Purchaser Trend - Volume

In [45]:
sales_size_screen_2 = sales_size_screen[sales_size_screen["qty_sum"] > 1]

In [46]:
YYYYMM_sorted = sorted(sales_size_screen.YYYYMM.unique())
YYYYMM = YYYYMM_sorted[-7:]

In [47]:
YYYYMM

['202308', '202309', '202310', '202311', '202312', '202401', '202402']

#### 0) By HnB

In [48]:
cigatype = ["HnB"]


# 브랜드별 월간 데이터를 담을 딕셔너리
HnB_monthly_dict = {}

for HnB in cigatype:
    # 월간 데이터를 담을 데이터프레임 생성 (cigatype를 반복하면서 monthly_list라는 빈 데이터프레임을 생성)
    monthly_list = pd.DataFrame(columns=["purchaser"])
    for yyyymm in YYYYMM:
        # 해당 월에 구매한 사용자 목록을 필터링

        # user_3month_list에서 해당월(yyyymm) 유효사용자에 해당하는 id를 기준으로 sales screen 데이터 필터링
        sales_3month_screen = sales_size_screen_2[
            sales_size_screen_2["id"].isin(
                user_3month_list[user_3month_list["YYYYMM"] == yyyymm]["id"]
            )
        ]
        # 필터링된 데이터에서 해당월에 해당하는 데이터만 저장
        sales_3month_screen = sales_3month_screen[
            sales_3month_screen["YYYYMM"] == yyyymm
        ]
        # 해당 월에 해당 브랜드를 구매한 고객 수를 계산하여 데이터프레임에 추가
        sales_df = pd.DataFrame(
            (
                sales_3month_screen[sales_3month_screen["CIGATYPE"] == HnB]
                .groupby("YYYYMM")["id"]
                .nunique()
            )
        ).rename(columns={"id": "purchaser"})
        # 고객 수 데이터를 월별로 합침
        monthly_list = pd.concat([monthly_list, sales_df])
    # 해당 브랜드의 월간 데이터를 딕셔너리에 저장
    HnB_monthly_dict[HnB] = monthly_list

In [49]:
HnB_monthly_dict["HnB"]  # new

,purchaser
202308,46495
202309,46855
202310,48060
202311,47353
202312,47090
202401,46877
202402,46475


In [50]:
# HnB_monthly_dict["HnB"].to_excel("HnB_monthly_dict.xlsx")

#### 1) By  TMO / ProductFamilyCode

In [51]:
brand_list = ["IQOS", "LIL", "GLO"]

# 브랜드별 월간 데이터를 담을 딕셔너리
brand_monthly_dict = {}

for brand in brand_list:
    # 월간 데이터를 담을 데이터프레임 생성 (brand_list를 반복하면서 monthly_list라는 빈 데이터프레임을 생성)
    monthly_list = pd.DataFrame(columns=["purchaser"])
    for yyyymm in YYYYMM:
        # 해당 월에 구매한 사용자 목록을 필터링

        # user_3month_list에서 해당월(yyyymm) 유효사용자에 해당하는 id를 기준으로 sales screen 데이터 필터링
        sales_3month_screen = sales_size_screen_2[
            sales_size_screen_2["id"].isin(
                user_3month_list[user_3month_list["YYYYMM"] == yyyymm]["id"]
            )
        ]
        # 필터링된 데이터에서 해당월에 해당하는 데이터만 저장
        sales_3month_screen = sales_3month_screen[
            sales_3month_screen["YYYYMM"] == yyyymm
        ]
        # 해당 월에 해당 브랜드를 구매한 고객 수를 계산하여 데이터프레임에 추가
        sales_df = pd.DataFrame(
            (
                sales_3month_screen[sales_3month_screen["ProductFamilyCode"] == brand]
                .groupby("YYYYMM")["id"]
                .nunique()
            )
        ).rename(columns={"id": "purchaser"})
        # 고객 수 데이터를 월별로 합침
        monthly_list = pd.concat([monthly_list, sales_df])
    # 해당 브랜드의 월간 데이터를 딕셔너리에 저장
    brand_monthly_dict[brand] = monthly_list

In [52]:
brand_monthly_dict  # new

{'IQOS':        purchaser
 202308     18342
 202309     18685
 202310     19156
 202311     18962
 202312     19020
 202401     18931
 202402     18914,
 'LIL':        purchaser
 202308     23623
 202309     23802
 202310     24220
 202311     23726
 202312     23515
 202401     23504
 202402     23306,
 'GLO':        purchaser
 202308      6264
 202309      6082
 202310      6458
 202311      6376
 202312      6303
 202401      6120
 202402      5966}

#### 2) By Brand / ProductSubFamilyCode

In [53]:
sales_size_screen_2[sales_size_screen_2["CIGATYPE"] == "HnB"][
    "ProductSubFamilyCode"
].unique()

array(['MIIX', 'HEETS', 'TEREA', 'NEOSTICKS', 'AIIM', 'NEO', 'FIIT'],
      dtype=object)

In [54]:
sub_brand_list = ["HEETS", "TEREA", "FIIT", "MIIX", "AIIM"]


# 서브 브랜드별 월간 데이터를 담을 딕셔너리
sub_brand_monthly_dict = {}

for sub_brand in sub_brand_list:
    # 월간 데이터를 담을 데이터프레임 생성
    monthly_list = pd.DataFrame(columns=["purchaser"])
    for yyyymm in YYYYMM:
        # 해당 월에 구매한 사용자 목록을 필터링
        sales_3month_screen = sales_size_screen_2[
            sales_size_screen_2["id"].isin(
                user_3month_list[user_3month_list["YYYYMM"] == yyyymm]["id"]
            )
        ]
        sales_3month_screen = sales_3month_screen[
            sales_3month_screen["YYYYMM"] == yyyymm
        ]
        # 해당 월에 해당 서브 브랜드를 구매한 고객 수를 계산하여 데이터프레임에 추가
        sales_df = (
            sales_3month_screen[
                sales_3month_screen["ProductSubFamilyCode"] == sub_brand
            ]
            .groupby("YYYYMM")["id"]
            .nunique()
        )
        # sales_df의 첫 번째 인덱스(해당 월)에 해당하는 행에 구매자 수 값을 추가
        monthly_list.loc[sales_df.index[0]] = sales_df.values[0]
    # 해당 서브 브랜드의 월간 데이터를 딕셔너리에 저장
    sub_brand_monthly_dict[sub_brand] = monthly_list

In [55]:
sub_brand_monthly_dict  # new

{'HEETS':         purchaser
 202308       6866
 202309       6450
 202310       5905
 202311       5360
 202312       4827
 202401       4269
 202402       3880,
 'TEREA':         purchaser
 202308      12223
 202309      13090
 202310      14056
 202311      14322
 202312      14869
 202401      15260
 202402      15587,
 'FIIT':         purchaser
 202308       3862
 202309       3707
 202310       3627
 202311       3383
 202312       3285
 202401       3201
 202402       2992,
 'MIIX':         purchaser
 202308      17653
 202309      18060
 202310      18548
 202311      18211
 202312      18174
 202401      18291
 202402      18268,
 'AIIM':         purchaser
 202308       2741
 202309       2657
 202310       2615
 202311       2652
 202312       2535
 202401       2534
 202402       2532}

# 5-2. HNB Category | Consumer Profile by Brands

In [56]:
sales_3month_screen = sales_size_screen_2[
    sales_size_screen_2["id"].isin(
        user_3month_list[user_3month_list["YYYYMM"] == month]["id"]
    )
]
sales_3month_screen = sales_3month_screen[sales_3month_screen["YYYYMM"] == month]

In [57]:
## TEREA

terea_id = sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "TEREA"][
    "id"
].unique()

# TEREA를 이달에 구매한 사람의 unique한 id의 수
print(
    len(
        sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "TEREA"][
            "id"
        ].unique()
    )
)

# 위의 값을 demo가 있는 데이터프레임에 조건을 건 unique한 id의 수
print(len(month_history[month_history["id"].isin(terea_id)]["id"].unique()))

# 1번과 2번의 값이 같아야 함!!!!

# 이번 달에 TEREA를 구매한 사람의 demo 데이터프레임
TEREA_monthly_demo = month_history[month_history["id"].isin(terea_id)][
    ["id", "gender", "age"]
].drop_duplicates()

# Terea Monthly Age
TEREA_monthly_demo_age = TEREA_monthly_demo.age.value_counts().sort_index()
TEREA_monthly_demo_age = pd.DataFrame(TEREA_monthly_demo_age).reset_index()
# Terea Monthly Gender
TEREA_monthly_demo_gender = TEREA_monthly_demo.gender.value_counts().sort_index()
TEREA_monthly_demo_gender = pd.DataFrame(TEREA_monthly_demo_gender).reset_index()

print(TEREA_monthly_demo_age)
print(TEREA_monthly_demo_gender)

15587
15587
   age  count
0  20대   1532
1  30대   4059
2  40대   6058
3  50대   3220
4  60대    581
5  70대    108
6   미상     29
  gender  count
0      남  11473
1     미상      6
2      여   4108


In [58]:
## MIIX

MIIX_id = sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "MIIX"][
    "id"
].unique()

# MIIX를 이달에 구매한 사람의 unique한 id의 수
print(
    len(
        sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "MIIX"][
            "id"
        ].unique()
    )
)

# 위의 값을 demo가 있는 데이터프레임에 조건을 건 unique한 id의 수
print(len(month_history[month_history["id"].isin(MIIX_id)]["id"].unique()))

# 1번과 2번의 값이 같아야 함!!!!

# 이번 달에 MIIX를 구매한 사람의 demo 데이터프레임
MIIX_monthly_demo = month_history[month_history["id"].isin(MIIX_id)][
    ["id", "gender", "age"]
].drop_duplicates()

# MIIX Monthly Age
MIIX_monthly_demo_age = MIIX_monthly_demo.age.value_counts().sort_index()
MIIX_monthly_demo_age = pd.DataFrame(MIIX_monthly_demo_age).reset_index()
# MIIX Monthly Gender
MIIX_monthly_demo_gender = MIIX_monthly_demo.gender.value_counts().sort_index()
MIIX_monthly_demo_gender = pd.DataFrame(MIIX_monthly_demo_gender).reset_index()

print(MIIX_monthly_demo_age)
print(MIIX_monthly_demo_gender)

18268
18268
   age  count
0  20대   5171
1  30대   5177
2  40대   4867
3  50대   2393
4  60대    472
5  70대     85
6   미상    103
  gender  count
0      남  10981
1     미상      8
2      여   7279


In [59]:
## AIIM

AIIM_id = sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "AIIM"][
    "id"
].unique()

# AIIM를 이달에 구매한 사람의 unique한 id의 수
print(
    len(
        sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "AIIM"][
            "id"
        ].unique()
    )
)

# 위의 값을 demo가 있는 데이터프레임에 조건을 건 unique한 id의 수
print(len(month_history[month_history["id"].isin(AIIM_id)]["id"].unique()))

# 1번과 2번의 값이 같아야 함!!!!

# 이번 달에 AIIM를 구매한 사람의 demo 데이터프레임
AIIM_monthly_demo = month_history[month_history["id"].isin(AIIM_id)][
    ["id", "gender", "age"]
].drop_duplicates()

# AIIM Monthly Age
AIIM_monthly_demo_age = AIIM_monthly_demo.age.value_counts().sort_index()
AIIM_monthly_demo_age = pd.DataFrame(AIIM_monthly_demo_age).reset_index()

# AIIM Monthly Gender
AIIM_monthly_demo_gender = AIIM_monthly_demo.gender.value_counts().sort_index()
AIIM_monthly_demo_gender = pd.DataFrame(AIIM_monthly_demo_gender).reset_index()

print(AIIM_monthly_demo_age)
print(AIIM_monthly_demo_gender)

2532
2532
   age  count
0  20대    457
1  30대    659
2  40대    875
3  50대    436
4  60대     82
5  70대     13
6   미상     10
  gender  count
0      남   1801
1      여    731


In [60]:
## FIIT

FIIT_id = sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "FIIT"][
    "id"
].unique()

# FIIT를 이달에 구매한 사람의 unique한 id의 수
print(
    len(
        sales_3month_screen[sales_3month_screen["ProductSubFamilyCode"] == "FIIT"][
            "id"
        ].unique()
    )
)

# 위의 값을 demo가 있는 데이터프레임에 조건을 건 unique한 id의 수
print(len(month_history[month_history["id"].isin(FIIT_id)]["id"].unique()))

# 1번과 2번의 값이 같아야 함!!!!

# 이번 달에 FIIT를 구매한 사람의 demo 데이터프레임
FIIT_monthly_demo = month_history[month_history["id"].isin(FIIT_id)][
    ["id", "gender", "age"]
].drop_duplicates()

# FIIT Monthly Age
FIIT_monthly_demo_age = FIIT_monthly_demo.age.value_counts().sort_index()
FIIT_monthly_demo_age = pd.DataFrame(FIIT_monthly_demo_age).reset_index()
# FIIT Monthly Gender
FIIT_monthly_demo_gender = FIIT_monthly_demo.gender.value_counts().sort_index()
FIIT_monthly_demo_gender = pd.DataFrame(FIIT_monthly_demo_gender).reset_index()

print(FIIT_monthly_demo_age)
print(FIIT_monthly_demo_gender)

2992
2992
   age  count
0  20대    285
1  30대    729
2  40대   1271
3  50대    584
4  60대     98
5  70대     22
6   미상      3
  gender  count
0      남   1885
1      여   1107


In [61]:
## GLO

GLO_id = sales_3month_screen[sales_3month_screen["ProductFamilyCode"] == "GLO"][
    "id"
].unique()

# GLO를 이달에 구매한 사람의 unique한 id의 수
print(
    len(
        sales_3month_screen[sales_3month_screen["ProductFamilyCode"] == "GLO"][
            "id"
        ].unique()
    )
)

# 위의 값을 demo가 있는 데이터프레임에 조건을 건 unique한 id의 수
print(len(month_history[month_history["id"].isin(GLO_id)]["id"].unique()))

# 1번과 2번의 값이 같아야 함!!!!

# 이번 달에 GLO를 구매한 사람의 demo 데이터프레임
GLO_monthly_demo = month_history[month_history["id"].isin(GLO_id)][
    ["id", "gender", "age"]
].drop_duplicates()

# GLO Monthly Age
GLO_monthly_demo_age = GLO_monthly_demo.age.value_counts().sort_index()
GLO_monthly_demo_age = pd.DataFrame(GLO_monthly_demo_age).reset_index()
# GLO Monthly Gender
GLO_monthly_demo_gender = GLO_monthly_demo.gender.value_counts().sort_index()
GLO_monthly_demo_gender = pd.DataFrame(GLO_monthly_demo_gender).reset_index()

print(GLO_monthly_demo_age)
print(GLO_monthly_demo_gender)

5966


5966
   age  count
0  20대    460
1  30대   1309
2  40대   2676
3  50대   1293
4  60대    182
5  70대     37
6   미상      9
  gender  count
0      남   4600
1      여   1366


# 5-3. HNB Category | Total HNB Category New Purchaser Trend - Volume

In [62]:
user_list_3month_anytobacco = user_3month_list.copy()
# 해당 월에 담배를 한번이라도 구입한 사람 중 지난 3개월 내에 담배를 구매했던 사람 id 를 user로 정의

user_list_3month_anytobacco["user"] = "user"
user_list_3month_anytobacco["YYYYMM"] = user_list_3month_anytobacco["YYYYMM"].astype(
    "str"
)

In [63]:
# check_3month 생성

check_3month = sales_size.merge(
    user_list_3month_anytobacco, how="left", on=["id", "YYYYMM"]
).set_index("YYYYMM")

# n_months 생성
n_months = check_3month.index.unique().sort_values()

# n_monthlist 생성
n_monthlist = [
    [a, b, c] for a, b, c in zip(n_months[:-12], n_months[11:], n_months[12:])
]
n_monthlist

[['202207', '202306', '202307'],
 ['202208', '202307', '202308'],
 ['202209', '202308', '202309'],
 ['202210', '202309', '202310'],
 ['202211', '202310', '202311'],
 ['202212', '202311', '202312'],
 ['202301', '202312', '202401'],
 ['202302', '202401', '202402']]

#### *TOTAL HnB

In [64]:
# cigatype에 있는 각 브랜드에 대해 작업 수행
cigatype = ["HnB"]

new_purchaser_list = pd.DataFrame()

for HnB in cigatype:
    for a, b, c in n_monthlist:
        # 지난 12개월 (a~b) HnB를 구매한 고객
        prev_df = check_3month[
            (check_3month["CIGATYPE"] == HnB)
            & (check_3month.index >= a)
            & (check_3month.index <= b)
        ]
        # 현재 달 (c)에 HnB를 구매한 고객
        df_month = check_3month[
            (check_3month["CIGATYPE"] == HnB)
            & (check_3month.index == c)
            & (check_3month["user"] == "user")
            & (check_3month.qty_sum > 1)
        ]

        # 현재월에 HnB를 구매했지만, 지난 12개월동안에는 HnB를 구매하지 않은 고객
        new_purchaser_df = df_month[~df_month["id"].isin(prev_df["id"])].copy()
        new_purchaser_df = len(new_purchaser_df["id"].unique())

        # dict 에 데이터 추가
        dict = {
            "brand": [HnB],
            "YYYYMM": [c],
            "new_purchaser": [new_purchaser_df],
        }
        df = pd.DataFrame.from_dict(dict)
        new_purchaser_list = pd.concat([new_purchaser_list, df])

    # new_purchaser_list 출력
new_purchaser_list

,brand,YYYYMM,new_purchaser
0,HnB,202307,3824
0,HnB,202308,3665
0,HnB,202309,3853
0,HnB,202310,3808
0,HnB,202311,3698
0,HnB,202312,3751
0,HnB,202401,3720
0,HnB,202402,3668


#### *GLO ONLY

In [80]:
a, b, c = ["202302", "202401", "202402"]

In [81]:
# 지난 12개월 (a~b) "GLO"를 구매한 고객
prev_df = check_3month[
    (check_3month["ProductFamilyCode"] == "GLO")
    & (check_3month.index >= a)
    & (check_3month.index <= b)
]
# 현재 달 (c)에 "GLO"를 구매한 고객
df_month = check_3month[
    (check_3month["ProductFamilyCode"] == "GLO")
    & (check_3month.index == c)
    & (check_3month["user"] == "user")
    & (check_3month.qty_sum > 1)
]

# 현재월에 "GLO"를 구매했지만, 지난 12개월동안에는 "GLO"를 구매하지 않은 고객
new_purchaser_df = df_month[~df_month["id"].isin(prev_df["id"])].copy()
new_purchaser_df = len(new_purchaser_df["id"].unique())
new_purchaser_df

630

#### *SUB-BRANDS ("TEREA", "MIIX", "FIIT", "AIIM")

In [82]:
n_monthlist

[['202207', '202306', '202307'],
 ['202208', '202307', '202308'],
 ['202209', '202308', '202309'],
 ['202210', '202309', '202310'],
 ['202211', '202310', '202311'],
 ['202212', '202311', '202312'],
 ['202301', '202312', '202401'],
 ['202302', '202401', '202402']]

In [68]:
# sub_brand_list에 있는 각 브랜드에 대해 작업 수행
sub_brand_list = ["TEREA", "MIIX", "FIIT", "AIIM"]

new_purchaser_list = pd.DataFrame()

for sub_brand in sub_brand_list:
    for a, b, c in n_monthlist:
        # 지난 12개월 (a~b) sub_brand를 구매한 고객
        prev_df = check_3month[
            (check_3month["ProductSubFamilyCode"] == sub_brand)
            & (check_3month.index >= a)
            & (check_3month.index <= b)
        ]
        # 현재 달 (c)에 sub_brand를 구매한 고객
        df_month = check_3month[
            (check_3month["ProductSubFamilyCode"] == sub_brand)
            & (check_3month.index == c)
            & (check_3month["user"] == "user")
            & (check_3month.qty_sum > 1)
        ]

        # 현재월에 sub_brand를 구매했지만, 지난 12개월동안에는 sub_brand를 구매하지 않은 고객
        new_purchaser_df = df_month[~df_month["id"].isin(prev_df["id"])].copy()
        new_purchaser_df = len(new_purchaser_df["id"].unique())

        # dict 에 데이터 추가
        dict = {
            "brand": [sub_brand],
            "YYYYMM": [c],
            "new_purchaser": [new_purchaser_df],
        }
        df = pd.DataFrame.from_dict(dict)
        new_purchaser_list = pd.concat([new_purchaser_list, df])

    # new_purchaser_list 출력
new_purchaser_list

,brand,YYYYMM,new_purchaser
0,TEREA,202307,2224
0,TEREA,202308,2156
0,TEREA,202309,2431
0,TEREA,202310,2298
0,TEREA,202311,2183
0,TEREA,202312,2150
0,TEREA,202401,2053
0,TEREA,202402,1976
0,MIIX,202307,2276
0,MIIX,202308,2319


# 기타 (아이코스 valid user / Acquisition , Retention 등)

In [69]:
# # qty_sum에 따른 비율 변화

# listfor = []

# for a in ["Normal", "morethan1", "morethan2", "morethan3"]:
#     for i in range(3):
#         listfor.append(a)

# ratiot = pd.concat(
#     [
#         ratio.groupby(["class"], as_index=False).count()[["class", "id"]],
#         ratio[ratio["qty_sum"] > 1]
#         .groupby(["class"], as_index=False)
#         .count()[["class", "id"]],
#         ratio[ratio["qty_sum"] > 2]
#         .groupby(["class"], as_index=False)
#         .count()[["class", "id"]],
#         ratio[ratio["qty_sum"] > 3]
#         .groupby(["class"], as_index=False)
#         .count()[["class", "id"]],
#     ]
# )
# ratiot.index = listfor
# ratiot = ratiot.reset_index().groupby(["index", "class"]).sum().unstack()
# ratiot.columns = ["Converted", "Predominant", "Situational"]
# ratiot["sum"] = ratiot.sum(axis=1)
# ratiot["situ_ratio"] = ratiot["Situational"] / ratiot["sum"] * 100
# ratiot["pred_ratio"] = ratiot["Predominant"] / ratiot["sum"] * 100
# ratiot["conv_ratio"] = ratiot["Converted"] / ratiot["sum"] * 100
# ratiot

### 아이코스 valid user / Acquisition , Retention 비율

In [70]:
# iqos를 구매한 적 있는 vailid smoker : user
check_iqos_3month = (
    sales_size[sales_size.ProductFamilyCode == "IQOS"]
    .merge(
        user_list_3month_anytobacco,
        how="left",
        right_on=["id", "YYYYMM"],
        left_on=["id", "YYYYMM"],
    )
    .fillna("no")
    .set_index("YYYYMM")
)

In [71]:
indexlist = sorted(check_iqos_3month.index.unique().to_list())
indexlistnew = [
    [a, b, c] for a, b, c in zip(indexlist[:-12], indexlist[11:], indexlist[12:])
]
indexlistnew

[['202207', '202306', '202307'],
 ['202208', '202307', '202308'],
 ['202209', '202308', '202309'],
 ['202210', '202309', '202310'],
 ['202211', '202310', '202311'],
 ['202212', '202311', '202312'],
 ['202301', '202312', '202401'],
 ['202302', '202401', '202402']]

In [72]:
user_list_aq = pd.DataFrame()
aq_list = []
rt_list = []

for a, b, c in indexlistnew:
    # 직전 12개월동안 아이코스 스틱 구매한 사람 필터링
    prev_consumer = check_iqos_3month[
        (check_iqos_3month.index >= a) & (check_iqos_3month.index <= b)
    ]
    # 해당 월에 아이코스 스틱 구매한 사람 필터링
    now_consumer = check_iqos_3month[check_iqos_3month.index == c]

    # new_consumer : 직전 12개월동안은 구매하지 않았지만, 해당 월에는 구매한 사람
    new_consumer = now_consumer[~now_consumer["id"].isin(prev_consumer["id"])]

    # old_consumer : 직전 12개월동안 구매했고, 해당 월에도 구매한 사람
    old_consumer = now_consumer[now_consumer["id"].isin(prev_consumer["id"])]

    # 직전 12개월동안은 구매하지 않았지만, 해당 월에는 구매한 사람 중 valid user
    user_list_aq = pd.concat([user_list_aq, new_consumer[new_consumer.user == "user"]])

    # 직전 12개월동안은 구매하지 않았지만, 해당 월에는 구매한 사람 중 unique한 valid user
    aq_list.append(len(new_consumer[new_consumer.user == "user"].id.unique()))

    # 전 12개월동안 구매했고, 해당 월에도 구매한 사람 중 unique한 valid user
    rt_list.append(len(old_consumer[old_consumer.user == "user"].id.unique()))

# 결과 데이터프레임 만들기

# YYYYMM, Acquisition, Retention, Total, Ac
result_df = pd.DataFrame(
    {
        "YYYYMM": [c for a, b, c in indexlistnew],
        "Aquisition": aq_list,
        "Retention": rt_list,
    }
).set_index("YYYYMM")

result_df["Total"] = result_df["Aquisition"] + result_df["Retention"]
result_df["AQ_per"] = result_df["Aquisition"] / (result_df["Total"]) * 100
result_df["RT_per"] = result_df["Retention"] / (result_df["Total"]) * 100

In [73]:
result_df

,Aquisition,Retention,Total,AQ_per,RT_per
YYYYMM,,,,,
202307,2192,19219,21411,10.24,89.76
202308,2140,19287,21427,9.99,90.01
202309,2366,19575,21941,10.78,89.22
202310,2229,20282,22511,9.90,90.10
202311,2225,20199,22424,9.92,90.08
202312,2058,20315,22373,9.20,90.80
202401,2067,20130,22197,9.31,90.69
202402,2006,20222,22228,9.02,90.98


In [74]:
#### 파일로 저장
# result_df.to_csv(

#     r"C:\Users\gchoi4\Philip Morris International\PMK_Finance_BP - Finance - korea - CX_team_collabo\L_point_data\Document\AQRT\aqrt_nov.csv"

# )

#### 12month aqrt

In [75]:
aqlist = new_consumer[new_consumer["user"] == "user"]
rtlist = old_consumer[old_consumer["user"] == "user"]

In [76]:
aqlist.groupby(["id", "qty_sum"], as_index=False).sum()

,id,qty_sum,CIGATYPE,ProductFamilyCode,ProductSubFamilyCode,ENGNAME,qty,sku,user
0,000B8571F847D61811A7A1ECFDBB60CA884DA6E62CB1AD...,2.00,HnB,IQOS,HEETS,HEETS SUMMER BREEZE,1.00,2,user
1,003115087663368965614E79A4D1CECE51B8D75C4D0084...,1.00,HnB,IQOS,TEREA,TEREA PURPLE WAVE,1.00,1,user
2,0031487845B039AF1FCA7E6F46DDBF14333AC763D1366F...,3.00,HnB,IQOS,TEREA,TEREA SILVER,2.00,2,user
3,003C364B9342289AC070515CDBCD4B1AF2D39841024CAF...,14.00,HnB,IQOS,TEREA,TEREA RUSSET,1.00,2,user
4,003DBE7F22F62A87CA5CADFDA96AEED1F44170CBE0C0CB...,1.00,HnB,IQOS,TEREA,TEREA PURPLE WAVE,1.00,1,user
...,...,...,...,...,...,...,...,...,...
2001,FFC11D43806D35280DBB258CAF4523ECBE8A3E1347D4E5...,7.00,HnB,IQOS,TEREA,TEREA BLUE,1.00,2,user
2002,FFC19AB82C6E9B74E9E2463E1A6E4B149DE2784C7C314B...,4.00,HnB,IQOS,TEREA,TEREA BLUE,1.00,4,user
2003,FFC519BF6709376873CF6A222299959443509E9402CA9A...,5.00,HnB,IQOS,TEREA,TEREA GREEN,1.00,3,user
2004,FFE5BDE8C0C065E4BCECA5A0E03278220D634CF92714C8...,4.00,HnB,IQOS,TEREA,TEREA GREEN,1.00,2,user


In [77]:
# aqlist 데이터프레임을 "id"와 "qty_sum" 열을 기준으로 그룹화하고, "qty" 열의 합을 계산하여 새로운 데이터프레임 생성
aq_conversion = aqlist.groupby(["id", "qty_sum"], as_index=False).sum()[
    ["id", "qty_sum", "qty"]
]
# "qty"와 "qty_sum" 열을 사용하여 "ratio" 열 계산
aq_conversion["ratio"] = aq_conversion["qty"] / aq_conversion["qty_sum"]

# "ratio" 값에 따라 "class" 열 생성
aq_conversion["class"] = np.where(
    aq_conversion["ratio"] >= 0.95,
    "Converted",
    np.where(
        aq_conversion["ratio"] >= 0.7,
        "Predominant",
        np.where(aq_conversion["ratio"] >= 0.0001, "Situational", "Non-user"),
    ),
)
# "class"를 기준으로 그룹화하여 각 클래스별로 "id" 개수 계산 및 "class"와 "id" 열을 선택하여 새로운 데이터프레임 생성
aq_ratio = aq_conversion.groupby(["class"], as_index=False).count()[["class", "id"]]
# "spec" 열에 "Normal" 값을 할당
aq_ratio["spec"] = "Normal"
# "spec"와 "class" 열을 기준으로 피벗 테이블 생성
aq_ratio = aq_ratio.pivot_table(index="spec", columns="class", values="id")

# qty_sum > 1 을 index로 하는 테이블 생성
aq_ratio1 = (
    aq_conversion[aq_conversion["qty_sum"] > 1]
    .groupby(["class"], as_index=False)
    .count()[["class", "id"]]
)
aq_ratio1["spec"] = "more than 1"
aq_ratio1 = aq_ratio1.pivot_table(index="spec", columns="class", values="id")
# Normal 이랑 qty_sum > 1 합치기
aq_ratio = pd.concat([aq_ratio, aq_ratio1])

In [78]:
aq_ratio["sum"] = aq_ratio.sum(axis=1)
aq_ratio["situ_ratio"] = aq_ratio["Situational"] / aq_ratio["sum"] * 100
aq_ratio["pred_ratio"] = aq_ratio["Predominant"] / aq_ratio["sum"] * 100
aq_ratio["conv_ratio"] = aq_ratio["Converted"] / aq_ratio["sum"] * 100
aq_ratio

class,Converted,Predominant,Situational,sum,situ_ratio,pred_ratio,conv_ratio
spec,,,,,,,
Normal,635.00,66.00,1305.00,2006.00,65.05,3.29,31.66
more than 1,351.00,66.00,1305.00,1722.00,75.78,3.83,20.38


In [79]:
# aq_ratio.to_excel(
#     r"C:\Users\kkim60\Philip Morris International\PMK_Finance_BP - Finance - korea - M11. PMK Power BI(Support)\CX_team_collabo\L_point_data\hnb_aqrt\231116_oct_basic_infoNuser\aq_oct.xlsx"
# )